In [22]:
import pandas as pd, numpy as np, os, warnings
warnings.filterwarnings('ignore')

# Define base paths for each asset
base_paths = {
    'EURUSD': r'C:\Users\jessi\OneDrive\Projects\portfolio_management\database\data\raw\tick_data\download\eurusd_tick_data.h5',
    'SPX': r'C:\Users\jessi\OneDrive\Projects\portfolio_management\database\data\raw\tick_data\download\usa500idxusd_tick_data.h5',
    'GOLD': r'C:\Users\jessi\OneDrive\Projects\portfolio_management\database\data\raw\tick_data\download\xauusd_tick_data.h5'
}

def load_tick_data(h5_path, symbol):
    dfs = []
    with pd.HDFStore(h5_path, mode='r') as store:
        for year in ['y2015']:
            for month in [f'm{str(m).zfill(2)}' for m in range(1, 13)]:
                for day in [f'd{str(d).zfill(2)}' for d in range(1, 32)]:
                    key = f'/{symbol}/{year}/{month}/{day}'
                    if key in store.keys():
                        df = store[key]
                        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                        df.set_index('timestamp', inplace=True)
                        df['mid'] = (df['askPrice'] + df['bidPrice']) / 2
                        dfs.append(df[['mid']])
    return pd.concat(dfs).sort_index()

# Load tick data for each asset
eurusd_df = load_tick_data(base_paths['EURUSD'], 'eurusd')
spx_df = load_tick_data(base_paths['SPX'], 'usa500idxusd')
gold_df = load_tick_data(base_paths['GOLD'], 'xauusd')

# Resample to daily frequency using last available price
eurusd_daily = eurusd_df['mid'].resample('1D').last()
spx_daily = spx_df['mid'].resample('1D').last()
gold_daily = gold_df['mid'].resample('1D').last()

# Combine into single DataFrame
prices = pd.concat([eurusd_daily, spx_daily, gold_daily], axis=1)
prices.columns = ['EURUSD', 'SPX', 'GOLD']

# Compute daily log returns
rets = np.log(prices).diff().dropna()
print(rets.tail(3))

              EURUSD       SPX      GOLD
timestamp                               
2015-12-29 -0.004443  0.011363  0.000675
2015-12-30  0.000851 -0.008199 -0.006714
2015-12-31 -0.007034 -0.002425 -0.000689


In [23]:
# FETCH DATA FROM YFINANCE AND CALCULATE LOG RETURNS

import pandas as pd, numpy as np, yfinance as yf, warnings
warnings.filterwarnings('ignore')

tickers = ['EURUSD=X', '^GSPC', 'GC=F']
data = yf.download(tickers, start='2015-01-01', end='2017-12-31')

# Use 'Adj Close' if available, otherwise fallback to 'Close'
if 'Adj Close' in data.columns:
    prices = data['Adj Close']
else:
    prices = data['Close']

prices.columns = ['EURUSD', 'SPX', 'GOLD']
rets = np.log(prices).diff().dropna()
print(rets.tail(3))


[*********************100%***********************]  3 of 3 completed

              EURUSD       SPX      GOLD
Date                                    
2017-12-27 -0.001234  0.002256  0.000791
2017-12-28  0.003611  0.005502  0.001832
2017-12-29  0.003433  0.009383 -0.005197


In [25]:
# ================================================================
# 2. FACTOR DATA
# ================================================================

from pandas_datareader import data as pdr
from sklearn.decomposition import PCA

term = pdr.get_data_fred('T10Y2Y', start='2015-01-01', end='2017-12-31')
dxy  = pdr.get_data_fred('DTWEXBGS', start='2015-01-01', end='2017-12-31').pct_change()
pca = PCA(n_components=1)
pc1 = pd.DataFrame(pca.fit_transform(rets), index=rets.index, columns=['PC1'])

factors = pd.concat([term, dxy, pc1], axis=1).ffill()
factors.columns = ['TERM', 'DXY', 'PC1']
common_idx = rets.index.intersection(factors.index)
rets, factors = rets.loc[common_idx], factors.loc[common_idx]

In [34]:
rets

,EURUSD,SPX,GOLD
2015-01-05,-0.011897,0.014980,-0.018447
2015-01-06,-0.000621,0.012711,-0.008933
2015-01-07,-0.005346,-0.007161,0.011563
2015-01-08,-0.003320,-0.001819,0.017730
2015-01-09,-0.003379,0.006270,-0.008439
...,...,...,...
2017-12-21,0.003107,0.000947,0.001984
2017-12-22,-0.001756,0.006371,-0.000458
2017-12-27,-0.001234,0.002256,0.000791
2017-12-28,0.003611,0.005502,0.001832


In [35]:
# ================================================================
# 3. REGRESS → BETA (Time-Series OLS per asset)
# ================================================================
import statsmodels.api as sm

def fit_beta(asset_ret, factor_df, window=252):
    y = asset_ret.iloc[-window:]
    X = sm.add_constant(factor_df.iloc[-window:])
    model = sm.OLS(y, X).fit()
    return model.params[1:], model.params[0], model.mse_resid  # beta, alpha, sigma_e^2

# Initialize beta matrix: rows = assets (n), columns = factors (k)
B = pd.DataFrame(index=rets.columns, columns=factors.columns)
alphas, idio_vars = {}, {}

# Loop over assets
for asset in rets.columns:
    beta, alpha, resid_var = fit_beta(rets[asset], factors)
    B.loc[asset] = beta  # Fill row for asset
    alphas[asset] = alpha * 252
    idio_vars[asset] = resid_var * 252

print("Beta Matrix B (n x k):\n", B.round(3))



Beta Matrix B (n x k):
             TERM       DXY       PC1
EURUSD -0.000462 -0.338852  0.053802
SPX    -0.000181 -0.200327  0.905926
GOLD   -0.000382 -0.396989 -0.428842


In [36]:
# ================================================================
# 4. FACTOR MODEL COVARIANCE
# ================================================================
Omega_f = factors.cov().values * 252  # Annualize factor covariance
Psi = np.diag(list(idio_vars.values()))
cov_factor = B.values @ Omega_f @ B.values.T + Psi

In [37]:
# ================================================================
# 5. LEDOIT-WOLF + HYBRID COVARIANCE
# ================================================================
from sklearn.covariance import LedoitWolf

def lw_cov_60d(date, returns=rets):
    window = returns.loc[:date].tail(60)
    return pd.DataFrame(LedoitWolf().fit(window).covariance_,
                        index=rets.columns, columns=rets.columns)

cov_lw = lw_cov_60d(rets.index[-1])
cov_final = 0.7 * cov_lw + 0.3 * pd.DataFrame(cov_factor, index=rets.columns, columns=rets.columns)

In [ ]:
# ================================================================
# 6. BLACK-LITTERMAN + MV WEIGHTS (Unconstrained, Shorting Allowed)
# ================================================================
risk_aversion = 1
n_assets = len(rets.columns)
mkt_w = pd.Series([1/n_assets]*n_assets, index=rets.columns)
pi = risk_aversion * cov_final @ mkt_w

P = np.array([[1, 0, 0],   # EURUSD = 6%
              [0, 1,  0]])  # SPX = 9%
Q = np.array([0.05, 0.09])
tau = 0.07
Omega = np.diag([tau * (P[i] @ cov_final @ P[i].T) for i in range(P.shape[0])])

inv_tau_S = np.linalg.inv(tau * cov_final.values.astype(float))
bl_mu = np.linalg.inv(inv_tau_S + P.T @ np.linalg.inv(Omega) @ P) @ \
        (inv_tau_S @ pi.values + P.T @ np.linalg.inv(Omega) @ Q)
bl_mu = pd.Series(bl_mu, index=rets.columns)

# MV Optimization (Unconstrained)
import cvxpy as cp
w = cp.Variable(n_assets)
ret = bl_mu.values @ w
risk = cp.quad_form(w, cov_final.values)
prob = cp.Problem(cp.Maximize(ret - risk_aversion * risk),
                  [cp.sum(w) == 1])  # Only budget constraint
prob.solve()
w_opt = pd.Series(w.value.round(3), index=rets.columns)
print("Optimal Weights:\n", w_opt)

In [ ]:
# ================================================================
# 7. REBALANCE ±5%
# ================================================================
current_w = w_opt.copy()
def needs_rebalance(curr, target, thresh=0.05):
    return ((curr - target).abs() > thresh * target).any()

In [ ]:
# ================================================================
# 8. BACKTEST (Walk-Forward)
# ================================================================
dates = pd.date_range('2018-01-01', '2025-11-07', freq='B')
nav = pd.Series(index=dates, dtype=float); nav.iloc[0] = 1.0
current_w = pd.Series([1/n_assets]*n_assets, index=rets.columns)

for i, d in enumerate(dates[1:], 1):
    if d not in rets.index:
        nav[d] = nav.iloc[i-1]
        continue

    # Re-estimate everything rolling
    # (repeat steps 3–6 with .loc[:d])
    # For brevity: assume w_new computed
    if needs_rebalance(current_w, w_new):
        current_w = w_new
    daily_ret = (current_w * rets.loc[d]).sum()
    nav[d] = nav.iloc[i-1] * (1 + daily_ret)

nav = nav.ffill()
print(f"CAGR: {((nav[-1]/nav[0])**(252/len(nav))-1)*100:.1f}%")